In [1]:
#|echo: false
import pandas as pd, numpy as np, matplotlib.pyplot as plt, altair as alt, pytz
from fastcore.all import *
from datetime import datetime, timedelta

## Get data

In [2]:
#|echo: false
print(f'Last execution time: {datetime.now(pytz.timezone("America/Lima")).strftime("%d/%m/%Y %T")}')

Last execution time: 29/06/2023 05:58:26


In [3]:
#|code-summary: Products type filter
explore_types = ['frutas', 'lacteos', 'verduras', 'embutidos', 'panaderia', 'desayuno', 'congelados', 'abarrotes',
                 'aves', 'carnes', 'pescados']

In [4]:
#|code-summary: Data table
path = Path('../../output')
csv_files = L(path.glob('*.csv')).filter(lambda o: os.stat(o).st_size>0)
pat_store = re.compile('(.+)\_\d+')
pat_date = re.compile('.+\_(\d+)')
df = (
    pd.concat([pd.read_csv(o).assign(store=pat_store.match(o.stem)[1], date=pat_date.match(o.stem)[1])
               for o in csv_files], ignore_index=True)
    .pipe(lambda d: d.assign(
        name=d.name.str.lower()+' ('+d.store+')',
        sku=d.id.where(d.sku.isna(), d.sku).astype(int),
        date=pd.to_datetime(d.date)
    ))
    .drop('id', axis=1)
    .loc[lambda d: d.category.str.contains('|'.join(explore_types))]
    # Filter products with recent data
    .loc[lambda d: d.name.isin(d.groupby('name').date.max().loc[ge(datetime.now()-timedelta(days=30))].index)]
    # Filter empty prices
    .loc[lambda d: d.price>0]
)
print(df.shape)
df.sample(3)

(520169, 8)


,sku,name,brand,category,uri,price,store,date
1385238,10614798,pancake & wafle hungry jack mix but caja 907g ...,HUNGRY JACK,https://www.plazavea.com.pe/panaderia-y-pastel...,https://www.plazavea.com.pe/pancakewafle-hungr...,23.9,plaza_vea,2023-03-09
557542,10406425,galletas sabrochas san jorge paquete 180g (pla...,SAN JORGE,https://www.plazavea.com.pe/abarrotes,NaN,4.5,plaza_vea,2022-10-27
515727,10585991,vinagre blanco fanny botella 500ml (plaza_vea),FANNY,https://www.plazavea.com.pe/abarrotes,https://www.plazavea.com.pe/vinagre-blanco-fan...,3.1,plaza_vea,2023-03-27


## Top changes (ratio)

In [5]:
top_changes = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift())/d.shift())
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes.head(3)

,store,sku,last_price,change,last_date
7645,plaza_vea,10023355,5.9,0.241195,2023-06-29
5322,plaza_vea,4024,21.9,-0.185874,2023-06-29
7646,plaza_vea,10023357,5.9,0.172282,2023-06-29


In [6]:
def plot_changes(df_changes, title):
    selection = alt.selection_point(fields=['name'], bind='legend')
    dff = df_changes.drop('change', axis=1).merge(df, on=['store','sku'])
    return (dff
     .pipe(alt.Chart)
     .mark_line(point=True)
     .encode(
         x='date',
         y='price',
         color=alt.Color('name').scale(domain=sorted(dff.name.unique().tolist())),
         tooltip=['name','price','last_price']
     )
     .add_params(selection)
     .transform_filter(selection)
     .interactive()
     .properties(width=650, title=title)
     .configure_legend(orient='top', columns=3)
    )

In [7]:
top_changes.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [8]:
(top_changes
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [9]:
(top_changes
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Top changes (absolute values)

In [10]:
top_changes_abs = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift()).iloc[-1])
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes_abs.head(3)

,store,sku,last_price,change,last_date
6598,plaza_vea,28404,79.9,30.02,2023-06-29
4372,plaza_vea,553,95.1,29.60,2023-06-29
7401,plaza_vea,83497,76.9,25.50,2023-06-29


In [11]:
top_changes_abs.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [12]:
(top_changes_abs
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [13]:
(top_changes_abs
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Search specific products

In [14]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*pollo)(?=.*entero).*') &
                ~df.name.str.contains(r'marinado|aderezo')].unique().tolist()
names

['pollo entero\xa0artisan\xa0libre de antibióticos x kg (plaza_vea)',
 'pollo entero fresco metro x kg (metro)',
 'pollo entero sin menudencia x kg (metro)',
 'pollo entero light  x kg (metro)']

In [15]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Pollo')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [16]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'palta') &
                ~df.name.str.contains(r'shampoo|humectante|vino|salsa|acondicionador|aceite')].unique().tolist()
names

['palta fuerte (plaza_vea)',
 'palta hass madura la caserita empaque 500g (plaza_vea)',
 "palta fuerte bell's madura (plaza_vea)",
 'palta fuerte metro x kg (metro)',
 'palta madura cremosita x kg (metro)',
 'palta hass x kg (metro)',
 'palta super fuerte x kg (metro)',
 'palta nava metro x kg (metro)']

In [17]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Palta')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [18]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*aceite)(?=.*vegetal)(?=.*900).*') &
                ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')].unique().tolist()
names

['aceite vegetal primor premium botella 900ml (plaza_vea)',
 'aceite vegetal deleite botella 900ml (plaza_vea)',
 "aceite vegetal bell's botella 900ml (plaza_vea)",
 'aceite vegetal cil botella 900ml (plaza_vea)',
 'aceite vegetal nicolini botella 900ml (plaza_vea)',
 'aceite vegetal cocinero botella 900ml (plaza_vea)',
 'aceite vegetal máxima 900ml (metro)',
 'aceite vegetal metro 900ml (metro)',
 'aceite vegetal cocinero 900ml (metro)',
 'aceite vegetal primor premium 900ml (metro)',
 'aceite vegetal primor clásico 900ml (metro)',
 'aceite vegetal deleite premium 900ml (metro)',
 'aceite vegetal cuisine&co botella 900 ml (metro)',
 'aceite vegetal primor clásico botella 900ml (plaza_vea)',
 'aceite vegetal del cielo botella 900ml (plaza_vea)']

In [19]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [20]:
#|echo: false
#|output: false
names = df.name[
    df.name.str.contains(r'(?=.*harina)(?=.*1kg).*')
#     & ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')
].unique().tolist()
names

['harina de maíz amarillo p.a.n. precocida bolsa 1kg (plaza_vea)',
 'harina de trigo nicolini sin preparar bolsa 1kg (plaza_vea)',
 'harina de arroz costeño bolsa 1kg (plaza_vea)',
 'harina sin preparar favorita paquete 1kg (plaza_vea)',
 'harina la casa marimel integral bolsa 1kg (plaza_vea)',
 'harina de trigo preparada blanca flor bolsa 1kg (plaza_vea)',
 'harina molitalia preparada bolsa 1kg (plaza_vea)',
 'harina preparada grano de oro paquete 1kg (plaza_vea)',
 'harina preparada favorita paquete 1kg (plaza_vea)',
 'harina de trigo sin preparar molitalia sin preparar bolsa 1kg (plaza_vea)',
 "harina de trigo sin preparar bell's bolsa 1kg (plaza_vea)",
 'harina sin preparar nicolini 1kg (metro)',
 'harina preparada blanca flor 1kg (metro)',
 'harina sin preparar favorita 1kg (metro)',
 'harina sin preparar blanca flor 1kg (metro)',
 'harina sin preparar molitalia 1kg (metro)',
 'harina preparada molitalia 1kg (metro)',
 'harina preparada favorita 1kg (metro)',
 'harina sin preparar

In [21]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)